Wednesday, June 21, 2023

My walkthrough of the notebook ...

[Chapter-2-Machine-Translation.ipynb](https://github.com/CRCTransformers/deepdive-book/blob/main/Chapter-2-Machine-Translation.ipynb)



### Machine Translation Comparison: Attention vs Transformer

In [1]:
import time
from datetime import date

startTime = time.time()
todaysDate = date.today()

Why do I always have so many issues with spacy??

!python -m spacy download fr  ... this NEVER works!

In [2]:
import spacy
import spacy.cli

In [3]:
spacyDownload = False
if spacyDownload:
    spacy.cli.download("fr_core_news_sm")

In [4]:
# !pip install ecco
# !pip install torchtext
# !pip install wget

# this would not work ... had to run outside of this container ...sigh
# %%capture
# !wget https://download.pytorch.org/tutorial/data.zip && unzip data.zip && rm data.zip

In [5]:
from io import open
import unicodedata
import string
import re
import random
import os

import torch
from torch import nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import Vocab, build_vocab_from_iterator
from collections import Counter 

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import ticker
import ecco
     

In [6]:
MAX_SENTENCE_LENGTH = 20
FILTER_TO_BASIC_PREFIXES = False
SAVE_DIR = os.path.join(".", "models")

ENCODER_EMBEDDING_DIM = 256
ENCODER_HIDDEN_SIZE = 256
DECODER_EMBEDDING_DIM = 256
DECODER_HIDDEN_SIZE = 256

In [7]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    !nvidia-smi
else:
    device = torch.device("cpu")
device

Wed Jun 21 19:23:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
|  0%   57C    P0    N/A /  70W |    454MiB /  2048MiB |      8%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:02:00.0 Off |                  N/A |
| 24%   

device(type='cuda')

In [8]:
with open('data/eng-fra.txt', encoding="utf-8") as f:
    lines = f.read().strip().split("\n")

In [9]:
print(f"{len(lines):,} English-French phrase pairs.\n")
print("~~~~~ Examples: ~~~~~")
for example in random.choices(lines, k=5):
    pair = example.split('\t')
    print(f"English:  {pair[0]}")
    print(f"French:   {pair[1]}")
    print()

135,842 English-French phrase pairs.

~~~~~ Examples: ~~~~~
English:  I want to see if I can do that.
French:   Je veux voir si je peux le faire.

English:  We saw nothing strange.
French:   Nous ne vîmes rien d'étrange.

English:  There are a lot of dustballs under the couch.
French:   Il y a beaucoup de moutons sous le canapé.

English:  If you know the answer to this question, please tell me.
French:   Si vous connaissez la réponse à cette question, dites-la-moi, s'il vous plait.

English:  What's the world's highest mountain?
French:   Quelle est la montagne la plus haute au monde ?



In [10]:
def unicodeToAscii(s):
    return ''.join(
      c for c in unicodedata.normalize('NFD', s) 
      if unicodedata.category(c) != 'Mn'
    )

In [11]:
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"[^a-zA-Z.!?]+", " ", s)
    return s

In [12]:
def filterPair(p, max_length, prefixes):
    good_length = (len(p[0].split(' ')) < max_length) and (len(p[1].split(' ')) < max_length)
    if len(prefixes) == 0:
        return good_length
    else:
        return good_length and p[0].startswith(prefixes)

In [13]:
def filterPairs(pairs, max_length, prefixes=()):
    return [pair for pair in pairs if filterPair(pair, max_length, prefixes)]

In [14]:
def prepareData(lines, filter=False, reverse=False, max_length=10, prefixes=()):
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    print(f"Given {len(pairs):,} sentence pairs.")

    if filter:
        pairs = filterPairs(pairs, max_length=max_length, prefixes=prefixes)
        print(f"After filtering, {len(pairs):,} remain.")

    return pairs

In [15]:
basic_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re ",
    'are you', 'am i ', 
    'were you', 'was i ', 
    'where are', 'where is',
    'what is', 'what are'
)

In [16]:
pairs = prepareData(lines, 
                    filter=True, 
                    max_length=MAX_SENTENCE_LENGTH, 
                    prefixes=basic_prefixes if FILTER_TO_BASIC_PREFIXES else ())

Given 135,842 sentence pairs.
After filtering, 135,284 remain.


The notebook from the authors repo shows 15, 593 remaining. Why do we get a different result?

In [19]:
pairs[:10]

[['go.', 'va !'],
 ['run!', 'cours !'],
 ['run!', 'courez !'],
 ['wow!', 'ca alors !'],
 ['fire!', 'au feu !'],
 ['help!', 'a l aide !'],
 ['jump.', 'saute.'],
 ['stop!', 'ca suffit !'],
 ['stop!', 'stop !'],
 ['stop!', 'arrete toi !']]

In [17]:
endTime = time.time()

elapsedTime = time.strftime("%H:%M:%S", time.gmtime(endTime - startTime))

print(todaysDate.strftime('# Run Date: %A, %B %d, %Y'))
print(f"# Run Time: {elapsedTime}")


# Run Date: Wednesday, June 21, 2023
# Run Time: 00:00:04
